In [1]:
import csv
import os
from multiprocessing import Pool
import requests
from bs4 import BeautifulSoup
import json

In [3]:
date_range = ['2010-01-01', '2019-12-31']

In [ ]:
def apply_multiprocessing(func, data):
    print('Start multiprocessing')
    pool = Pool(processes=os.cpu_count())
    result = pool.map(func, data)
    pool.close()

    return result

In [ ]:
def crawling(movie_str):
    movie = BeautifulSoup(movie_str, 'html.parser')
    tb_row = [x.text.strip().replace('\r', '').replace('\t', '').replace('\n', '') for x in movie.select('td span')]
    code = tb_row[2].replace(' ', '')
    movie_row = [tb_row[0], tb_row[1], code, tb_row[7]]
    print(movie_row)
    staff_rows = []

    response_staff = requests.post('http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovStaffLists.do',
                                   headers={"Accept": "application/json, text/javascript, */*; q=0.01"},
                                   data={'movieCd': f'{code}'})
    staffs = json.loads(response_staff.content.decode())
    directors = [(staff['peopleNm'], staff['peopleCd']) for staff in staffs if staff['roleCd'] == '320301']
    for director_name, director_id in directors:
        staff_rows.append((code, 'director', director_name, director_id))

    response_actor = requests.post('http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovActorLists.do',
                                   headers={"Accept": "application/json, text/javascript, */*; q=0.01"},
                                   data={'movieCd': f'{code}'})
    actors = json.loads(response_actor.content.decode())
    main_actor = [(actor['peopleNm'], actor['peopleCd']) for actor in actors if actor['actorGb'] == '1']
    for actor_name, actor_id in main_actor:
        staff_rows.append((code, 'actor', actor_name, actor_id))

    response_dtl = requests.get('http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieDtl.do',
                                params={'code': f'{code}'})
    soup_dtl = BeautifulSoup(response_dtl.content, 'html.parser')
    dtl_dts = soup_dtl.select('dl.ovf.cont dt')
    dtl_dts_text = [dt.text for dt in dtl_dts]
    dtl_dds = soup_dtl.select('dl.ovf.cont dd')

    if '요약정보' in dtl_dts_text:
        summary = dtl_dds[dtl_dts_text.index('요약정보')].text.split('|')
        genre = summary[2].strip().replace('\r', '').replace('\t', '').replace('\n', '')
        country = summary[-1].strip().replace('\r', '').replace('\t', '').replace('\n', '')
    else:
        genre = ''
        country = ''
    if '개봉일' in dtl_dts_text:
        open_date = dtl_dds[dtl_dts_text.index('개봉일')].text.strip().replace('\r', '').replace('\t', '').replace('\n', '')
    else:
        open_date = ''

    movie_row.append(country.replace(' ', ''))
    movie_row.append(genre.replace(' ', ''))
    movie_row.append(open_date.replace(' ', ''))

    return (movie_row, staff_rows)

In [40]:
def main():
    curpage = 1
    page_length = 1
    movies_str = []
    while page_length > 0:
        response = requests.get('http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do',
                                params={'curPage': curpage,
                                        'sOpenYearS': date_range[0],
                                        'sOpenYearE': date_range[1]})
        soup = BeautifulSoup(response.content, 'html.parser')
        movies = soup.select('tr')[1:]
        movies_str.extend([x.decode() for x in movies])
        page_length = len(movies)

        print(f'curpage: {curpage}')
        curpage += 1

    movie_f = open('movie_list.csv', 'w', encoding='UTF-8')
    staff_f = open('staff_list.csv', 'w', encoding='UTF-8')

    movie_writer = csv.writer(movie_f, lineterminator='\n')
    movie_writer.writerow(('title', 'title_en', 'code', 'status', 'country', 'genre', 'open'))

    staff_writer = csv.writer(staff_f, lineterminator='\n')
    staff_writer.writerow(('movie_id', 'person_category', 'person_name', 'person_id'))

    # py 파일일 때 실행
    movie_rows = apply_multiprocessing(crawling, movies_str)

    # ipynb 파일일 때 실행
    # movie_rows = []
    # for movie_str in movies_str:
    #     movie_rows.append(crawling(movie_str))

    for movie in movie_rows:
        movie_writer.writerow(movie[0])
        for staff_row in movie[1]:
            staff_writer.writerow(staff_row)
    movie_f.close()
    staff_f.close()

TypeError: unhashable type: 'list'

In [16]:
if __name__ == '__main__':
    main()

,movie_id,person_id.x,person_id.y
2465,20110221,D10010652,A10063816
2466,20110221,D10010652,A10030781
2467,20110221,D10010652,A10027183
